# Voila App Components - Demonstration Notebook

This notebook demonstrates **reusable code snippets** for building interactive Voila applications. Each section shows you how to use pre-built Python modules that you can copy and adapt for your own projects.

---

## What You'll Learn

This notebook contains four practical examples:

1. **Tab Creation** - How to organize your app into multiple tabs using `ipywidgets`
2. **Authentication** - How to connect to NOMAD using automatic token authentication
3. **Batch Selection** - How to create interactive dropdowns that fetch and display NOMAD batch data
4. **Resizable Plots** - How to create professional, resizable Plotly charts

Also remember
1. If you find an application that you want to adapt, make sure to copy and rename the containing folder and adapt it to your own needs
2. Don't be afraid of using the chatbot you trust the most

**💡 Each section is independent** - you can run them in any order and copy the code for your own applications.

**📁 Required Files** - Make sure these Python files are in your parent directory:
- `access_token.py`
- `batch_selection.py`
- `api_calls.py`
- `resizable_plot_utility.py`

## Setup: Install and Import Required Libraries

**What these libraries do:**
- `ipywidgets` - Creates interactive buttons, dropdowns, and tabs in Jupyter notebooks
- `plotly` - Creates interactive, publication-quality charts and graphs
- `requests` - Makes HTTP requests to communicate with web APIs (like NOMAD)
- `numpy` - Performs mathematical operations and handles numerical arrays
- `pandas` - Organizes and analyzes data in table format (needed by some modules)
- `openpyxl` - Reads and writes Excel files (needed by some modules)

**💡 Installation:** If any library is missing, uncomment the first line in the cell below and run it.

In [1]:
# Install required packages (uncomment if needed)
# !pip install ipywidgets plotly requests pandas openpyxl numpy

# Import base libraries
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import plotly.graph_objects as go
import plotly.io as pio
from plotly.offline import init_notebook_mode
import numpy as np
import os

# Initialize Plotly for notebook mode
init_notebook_mode(connected=True)
pio.renderers.default = 'notebook'

print("✅ Base libraries imported successfully")

✅ Base libraries imported successfully


---

# Section 1: Authentication with NOMAD

**What this section does:**  
Connects to the NOMAD database using your authentication token. This is required before you can load any data from NOMAD.

**How it works:**
1. Reads your authentication token from an environment variable (`NOMAD_CLIENT_ACCESS_TOKEN`)
2. Verifies the token is valid by making a test API call
3. Retrieves your user information (name, email)
4. Stores the token for use in later sections

**Required files:**
- None (this section uses only standard libraries)

**⚠️ Important:**  
You must be logged into NOMAD in your browser for this to work. The token is automatically set by NOMAD when you're logged in.

**📖 What you can learn from this code:**
- How to read environment variables in Python
- How to make authenticated API requests with the `requests` library
- How to handle authentication errors gracefully

In [2]:
# Import authentication modules (they are one folder up)
import sys
import os
import requests

# Add parent directory to path
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# NOMAD server configuration - use the correct HZB URL
NOMAD_URL = 'https://nomad-hzb-se.de/nomad-oasis/api/v1'

print("\n" + "="*70)
print("NOMAD AUTHENTICATION")
print("="*70)

# Global variable to store token
AUTHENTICATED_TOKEN = None
AUTHENTICATED_USER = None

# Get token from environment
print("\n🔄 Attempting authentication...")
try:
    token = os.environ.get('NOMAD_CLIENT_ACCESS_TOKEN')
    if not token:
        raise ValueError("NOMAD_CLIENT_ACCESS_TOKEN environment variable not set")
    
    # Verify the token works
    verify_url = f"{NOMAD_URL}/users/me"
    headers = {'Authorization': f'Bearer {token}'}
    response = requests.get(verify_url, headers=headers, timeout=10)
    response.raise_for_status()
    
    user_info = response.json()
    AUTHENTICATED_TOKEN = token
    AUTHENTICATED_USER = user_info.get('name', user_info.get('username', 'Unknown'))
    
    print(f"✅ User authenticated through NOMAD token")
    print(f"📋 Logged in as: {AUTHENTICATED_USER}")
    print(f"📧 Email: {user_info.get('email', 'N/A')}")
    print(f"🔑 Token (first 20 chars): {token[:20]}...")
    print("\n✅ Ready to proceed with data loading!")
    
except Exception as e:
    print(f"❌ Authentication failed: {e}")
    print("\n⚠️  Please ensure:")
    print("  - You are logged into NOMAD")
    print("  - NOMAD_CLIENT_ACCESS_TOKEN environment variable is set")
    print("  - You have network connectivity")


NOMAD AUTHENTICATION

🔄 Attempting authentication...
✅ User authenticated through NOMAD token
📋 Logged in as: Edgar Nandayapa
📧 Email: None
🔑 Token (first 20 chars): eyJhbGciOiJIUzI1NiIs...

✅ Ready to proceed with data loading!


---

# Section 2: Creating Tabbed Interfaces

**What this section does:**  
Shows you how to organize your application into multiple tabs, similar to how the JV Analysis app is structured.

**Why use tabs:**
- Keeps your interface organized and uncluttered
- Groups related functionality together
- Makes complex applications easier to navigate

**What's demonstrated:**
- Creating multiple tab contents with different widgets (text inputs, buttons, dropdowns)
- Making buttons functional with click handlers
- Displaying output when buttons are clicked

**📖 Key concepts to learn:**
- `widgets.Tab()` - The main container for tabbed content
- `widgets.VBox()` - Stacks widgets vertically
- `widgets.Output()` - Creates an area where you can print messages
- `.on_click()` - Attaches a function to run when a button is clicked

**💡 Try this:**  
After running this cell, click the buttons in different tabs to see how they respond!

In [4]:
# Create output areas for each tab
tab1_output = widgets.Output()
tab2_output = widgets.Output()
tab3_output = widgets.Output()

# Create buttons
auth_button = widgets.Button(description="Authenticate", button_style='primary')
load_button = widgets.Button(description="Load Data", button_style='success')
plot_button = widgets.Button(description="Generate Plot", button_style='info')

# Button click handlers
def on_auth_click(b):
    with tab1_output:
        clear_output()
        print("🔘 Authenticate button pressed!")

def on_load_click(b):
    with tab2_output:
        clear_output()
        print("🔘 Load Data button pressed!")

def on_plot_click(b):
    with tab3_output:
        clear_output()
        print("🔘 Generate Plot button pressed!")

# Attach handlers
auth_button.on_click(on_auth_click)
load_button.on_click(on_load_click)
plot_button.on_click(on_plot_click)

# Create content for each tab
tab1_content = widgets.VBox([
    widgets.HTML("<h3>Connection Settings</h3>"),
    widgets.Label("This tab would contain authentication widgets, it is just a dummy example"),
    widgets.Text(placeholder="Enter username", description="Username:"),
    widgets.Password(placeholder="Enter password", description="Password:"),
    auth_button,
    tab1_output
])

tab2_content = widgets.VBox([
    widgets.HTML("<h3>Batch Selection</h3>"),
    widgets.Label("This tab would contain batch selection widgets"),
    widgets.SelectMultiple(
        options=['Batch_001', 'Batch_002', 'Batch_003'],
        description='Batches:',
        layout=widgets.Layout(width='400px', height='200px')
    ),
    load_button,
    tab2_output
])

tab3_content = widgets.VBox([
    widgets.HTML("<h3>Data Analysis</h3>"),
    widgets.Label("This tab would contain analysis tools and plots"),
    widgets.Dropdown(
        options=['Boxplot', 'Histogram', 'JV Curve'],
        description='Plot Type:',
        value='Boxplot'
    ),
    plot_button,
    tab3_output
])

# Create the tab widget
tabs = widgets.Tab(children=[tab1_content, tab2_content, tab3_content])
tabs.set_title(0, 'Connection')
tabs.set_title(1, 'Batches')
tabs.set_title(2, 'Analysis')

# Display the tabs
display(tabs)
print("✅ Tabbed interface created successfully")
print("💡 Click on different tabs to switch between views")
print("💡 Try clicking the buttons in each tab!")

✅ Tabbed interface created successfully
💡 Click on different tabs to switch between views
💡 Try clicking the buttons in each tab!


---

# Section 3: Batch Selection Widget

**What this section does:**  
Creates an interactive dropdown that fetches batch names from NOMAD and lets users select which batches to load data from.

**How it works:**
1. Calls NOMAD API to get all available batch IDs
2. Creates a searchable multi-select widget showing all batches
3. When "Load Data" is clicked, prints which batches were selected

**Required files (in parent directory):**
- `batch_selection.py` - Creates the batch selection UI widget
- `api_calls.py` - Contains functions for querying NOMAD database

**Prerequisites:**
- ⚠️ You must run Section 1 (Authentication) first to get your NOMAD token

**📖 What you can learn from this code:**
- How to fetch data from an API and display it in a widget
- How to use `widgets.SelectMultiple` for multi-selection
- How to implement search/filter functionality
- How to pass callback functions to handle button clicks

**💡 In a real app:**  
Instead of printing selected batches, you would load the actual sample data and measurements for analysis.

In [5]:
# Import required modules
try:
    from batch_selection import create_batch_selection
    from api_calls import get_batch_ids
    print("✅ Batch selection modules imported successfully")
except ImportError as e:
    print(f"❌ Error importing batch selection modules: {e}")
    print("Make sure batch_selection.py and api_calls.py are in the same directory")

# Check if authenticated
try:
    token = AUTHENTICATED_TOKEN
    print(f"✅ Using authenticated token")
except NameError:
    print("⚠️  No authentication token found. Please run Section 2 first.")
    print("   For demo purposes, you can set a token manually:")
    print("   AUTHENTICATED_TOKEN = 'your_token_here'")
    token = None

# Output area for batch loading
batch_output = widgets.Output(
    layout=widgets.Layout(
        border='1px solid #ddd',
        padding='10px',
        margin='10px 0',
        min_height='100px'
    )
)

# Function to handle batch loading
def load_batch_data(batch_selector_widget):
    """Handler function called when Load Data button is clicked"""
    with batch_output:
        clear_output(wait=True)
        selected_batches = batch_selector_widget.value
        
        if not selected_batches:
            print("⚠️  No batches selected")
            return
        
        print(f"✅ Load Data button clicked!")
        print(f"\n📦 Selected Batches ({len(selected_batches)}):")
        for i, batch in enumerate(selected_batches, 1):
            print(f"  {i}. {batch}")
        
        print(f"\n💾 Next steps would be:")
        print(f"  - Query NOMAD for samples in these batches")
        print(f"  - Load JV measurement data")
        print(f"  - Process and display the data")

# Create the batch selection widget
if token:
    try:
        print("\n🔄 Fetching batch IDs from NOMAD...")
        batch_widget = create_batch_selection(NOMAD_URL, token, load_batch_data)
        print("✅ Batch selection widget created successfully")
        
        # Display the widget with output area
        display(widgets.VBox([
            widgets.HTML("<h3>Batch Selection</h3>"),
            widgets.HTML("<p>Search for batches and select multiple items using Ctrl/Cmd + Click</p>"),
            batch_widget,
            batch_output
        ]))
        
    except Exception as e:
        print(f"❌ Error creating batch selection widget: {e}")
        print("\nThis might be due to:")
        print("  - Invalid authentication token")
        print("  - Network connectivity issues")
        print("  - NOMAD server unavailable")
else:
    print("\n⚠️  Cannot create batch selection widget without authentication")
    print("Please authenticate first in Section 2")

print("\n💡 Instructions:")
print("  1. Use the search field to filter batches")
print("  2. Select one or more batches (Ctrl/Cmd + Click for multiple)")
print("  3. Click 'Load Data' to see selected batches")

✅ Batch selection modules imported successfully
✅ Using authenticated token

🔄 Fetching batch IDs from NOMAD...
✅ Batch selection widget created successfully



💡 Instructions:
  1. Use the search field to filter batches
  2. Select one or more batches (Ctrl/Cmd + Click for multiple)
  3. Click 'Load Data' to see selected batches


---

# Section 4: Creating Resizable Interactive Plots

**What this section does:**  
Shows you how to create professional Plotly charts that users can resize by dragging the corner, making your visualizations more flexible and user-friendly.

**Why use resizable plots:**
- Users can adjust plot size to their screen and preferences
- Better for presentations and reports where you need different sizes
- More professional appearance than fixed-size plots

**What's demonstrated:**
- Creating a line plot with multiple data series (Example 1)
- Creating a bar chart with color-coded values (Example 2)
- Using the `display_resizable_plot()` function

**Required files (in parent directory):**
- `resizable_plot_utility.py` - Contains the plot resizing functionality

**📖 Key function to use in your code:**
```python
display_resizable_plot(fig, title, width, height)
```
- `fig` - Your Plotly figure object
- `title` - Title to display above the plot
- `width` - Initial width in pixels (e.g., 800)
- `height` - Initial height in pixels (e.g., 500)

**💡 After running this cell:**  
Try dragging the bottom-right corner of each plot to resize it. The plot will automatically adjust!

In [6]:
import plotly.graph_objects as go
import numpy as np
from resizable_plot_utility import (
        ResizablePlotWidget,
        create_resizable_plot,
        display_resizable_plot,
        ResizablePlotManager)

# Example 1: Line Plot with resizable function
print("Creating Example 1: Trigonometric Functions...")
x = np.linspace(0, 10, 100)
y1 = np.sin(x) + np.random.normal(0, 0.1, 100)
y2 = np.cos(x) + np.random.normal(0, 0.1, 100)

fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=x, y=y1, mode='lines+markers', name='Sin Wave',
                          line=dict(color='#1f77b4', width=2), marker=dict(size=4)))
fig1.add_trace(go.Scatter(x=x, y=y2, mode='lines+markers', name='Cos Wave',
                          line=dict(color='#ff7f0e', width=2), marker=dict(size=4)))
fig1.update_layout(title='Trigonometric Functions with Noise', xaxis_title='X Values',
                   yaxis_title='Y Values', template="plotly_white", hovermode='x unified')

display_resizable_plot(fig1, "Example 1: Trigonometric Functions", 800, 500)
print("✅ Plot 1 created!\n")

# Example 2: Bar Chart
print("Creating Example 2: Efficiency Comparison...")
batches = ['Batch_A', 'Batch_B', 'Batch_C', 'Batch_D', 'Batch_E']
efficiency_values = [12.5, 14.2, 13.8, 15.1, 13.3]

fig2 = go.Figure()
fig2.add_trace(go.Bar(x=batches, y=efficiency_values,
                      marker=dict(color=efficiency_values, colorscale='Viridis'),
                      text=[f"{val:.1f}%" for val in efficiency_values], textposition='outside'))
fig2.update_layout(title='Power Conversion Efficiency by Batch', xaxis_title='Batch ID',
                   yaxis_title='Parameter (%)', template="plotly_white")

display_resizable_plot(fig2, "Example 2: Efficiency Comparison", 700, 500)
print("✅ Plot 2 created!\n")

print("\n✅ All example plots created successfully!")
print("💡 Drag the bottom-right corner of each plot to resize it")
print("💡 Use display_resizable_plot(fig, title, width, height) in your own code")

Creating Example 1: Trigonometric Functions...


✅ Plot 1 created!

Creating Example 2: Efficiency Comparison...


✅ Plot 2 created!


✅ All example plots created successfully!
💡 Drag the bottom-right corner of each plot to resize it
💡 Use display_resizable_plot(fig, title, width, height) in your own code


---

## Summary & Next Steps

**🎉 Congratulations!** You've learned the four essential building blocks for creating Voila applications:

### What You've Learned:

1. **✅ Tab Creation** - Organize complex UIs into manageable sections
   - Use `widgets.Tab()` to create tabbed interfaces
   - Connect buttons to functions with `.on_click()`

2. **✅ Authentication** - Connect securely to NOMAD database
   - Read environment variables with `os.environ.get()`
   - Make authenticated API requests with `requests` library

3. **✅ Batch Selection** - Create interactive data loading widgets
   - Use `batch_selection.py` to fetch and display NOMAD batches
   - Implement search and multi-select functionality

4. **✅ Resizable Plots** - Create professional, flexible visualizations
   - Use `display_resizable_plot(fig, title, width, height)` for any Plotly chart
   - Users can drag to resize plots in real-time
5. **✅ Use LLMs** - Using chatbots can increase the speed at which you can develop these apps

### Files You Can Reuse:
Copy these Python files to your own project:
- `batch_selection.py` - Batch selection widget
- `api_calls.py` - NOMAD API query functions
- `resizable_plot_utility.py` - Resizable plot functionality

### How to Build Your Own App:

1. **Start with a notebook** - Copy sections from this demo
2. **Add your data processing** - Load and analyze your specific data
3. **Create your visualizations** - Use Plotly to create charts
4. **Add interactivity** - Connect widgets to your analysis functions
5. **Deploy with Voila** - Run `voila your_notebook.ipynb` to create a web app

### Need Help?
- Check the Plotly documentation: https://plotly.com/python/
- ipywidgets guide: https://ipywidgets.readthedocs.io/
- Ask your colleagues who have built similar apps!

---

For questions or comments contacts edgar.nandayapa@helmholtz-berlin.de

Helmholtz-Zentrum Berlin - December 2025